In [64]:
import pandas as pd
import random
# from utils import *

In [65]:
#  List of names that need to br grammar-corrected
generational_suffixes = ['Jr.', 'Sr.', 'I', 'Ii', 'Iii', 'Iv', 'V', 'Vi', 'Vii', 'Viii', 'Ix', 'X']
special = ['J', 'K', 'Lequint', 'Mc', 'Rock']
double_name = ['Le', 'St.']
Upper = ['Aj', 'At', 'Bj', 'Cj', 'Dj', 'Dk', 'Ej', 'Jb', 'Jj', 'Jp', 'Jr', 'Jt', 'Kd', 'Kj', 'Kt', 'Ld', 'Lv', 'Mj', 'Oc', 'Pj', 'Rj', 'Tj']
Upper.extend(generational_suffixes)
Upper.remove('Jr.')
Upper.remove('Sr.')

first_count = {}
first_hyphen_total = 0
last_count = {}
last_hyphen_total = 0
suffix_count = {}
suffix_total = 0

#  Counting variables for name statistical weight and hyphen/suffix comminality 
def initiate_vars():
    global first_count
    global first_hyphen_total
    global last_count
    global last_hyphen_total
    global suffix_count
    global suffix_total

    first_count = {}
    first_hyphen_total = 0
    last_count = {}
    last_hyphen_total = 0
    suffix_count = {}
    suffix_total = 0

#  Fills the first_count and first_hyphen_total variables, calculating statistical weights for first names
def first_name(s):
    global first_count
    global first_hyphen_total
    if '-' not in s:
        # Non-hyphenated names default to a weight of 2 to lessen the chance of a non-unique hyphenated name
        first_count[s] = first_count.get(s, 99) + 1
    else:
        first_hyphen_total += 1
        #  Seperate hyphenated names since they are not last names
        s = s.split('-')
        first_count[s[0]] = first_count.get(s[0], 0) + 1
        first_count[s[-1]] = first_count.get(s[-1], 0) + 1

#  Fills the last_count and last_hyphen_total variables, calculating statistical weights for last names
def last_name(s):
    global last_count
    global last_hyphen_total
    if '-' not in s:
        # Non-hyphenated names default to a weight of 2 to lessen the chance of a non-unique hyphenated name
        last_count[s] = last_count.get(s, 99) + 1
    else:
        last_hyphen_total += 1
        #  Keep hyphenated names together for story line puropses
        last_count[s] = last_count.get(s, 0) + 1
        #  Also split the name and use both halves independantly in the dictionary 
        s = s.split('-')
        for name in s:
            last_name(name)

#  Fills the suffix_count and suffix_total variables, calculating statistical weights for suffixes
def suffix(s):
    global generational_suffixes
    global suffix_count
    global suffix_total
    if s not in generational_suffixes:
        last_name(s)
        return
    suffix_total += 1
    suffix_count[s] = suffix_count.get(s, 0) + 1

#  Special title to fix str.title()'s apostrophe problem
def special_title(s:str):
    s = s.title()
    return s

#  Changes special names
def check_special(s:str):
    if len(s) > 2 and s[-2] == "'":
        s = s[:-1] + s[-1].lower()
    if s[:2] == 'Mc':
        return 'Mc' + s[2:].title()
    if s in special:
        if s == 'J':
            return random_name(0)
        elif s == 'K':
            return random_name(0)
        elif s == 'Lequint':
            return 'LeQuint'
        elif s == 'Rock':
            return random_name(0)
    return s        

#  Will re-spin the name if there is more than one hyphen
def check_hyphenated_grammar(arr, i):
    ret = []
    #  Avoid using any more than one hyphen
    for name in arr:
        while "-" in name:
            name = random_name(i)
        ret.append(name)
    # Avoid using names that end in an apostrophe as teh first numae in a hyphenated name
    if ret[0][-1] == "'":
        ret[0] = ret[0][:-1]
    return ret

#  Returns a random first name with argument 0, last name with argument 1, and suffix with argument 2
def random_name(n:int = -1):
    if n == 0:
        return check_special(random.choices(list(first_count.keys()), weights = list(first_count.values()), k = 1)[0])
    elif n == 1:
        return check_special(random.choices(list(last_count.keys()), weights = list(last_count.values()), k = 1)[0])
    elif n == 2:
        return check_special(random.choices(list(suffix_count.keys()), weights = list(suffix_count.values()), k = 1)[0])
    return 'ERROR_NAME'

def get_name(total):
    name = ''
    ret = []
    total = float(total)
    first_hyphen_percentage = (first_hyphen_total / total) * 10.0
    last_hyphen_percentage = (last_hyphen_total / total) * 10.0
    suffix_hyphen_percentage = (suffix_total / total) * 10.0

    #^ -- Get First Name -- ^#
    #  If the name is hyphenated, run the first name randomizer twice
    if (random.randrange(100000) / 1000.0) < first_hyphen_percentage:
        name = []
        hyphenated = []
        hyphenated.append(random_name(0))
        hyphenated.append(random_name(0))
        #  Rerun if more than one hyphen
        hyphenated = check_hyphenated_grammar(hyphenated, 0)
        #  Check for grammar adjustments and form name in name array
        for t in hyphenated:
            if t in Upper:
                name.append(t.upper())
            else:
                name.append(t)
        name = '-'.join(name)
    #  If no hyphen is needed, run the first name randomizer once
    else:
        name = random_name(0)
        #  Check for grammar adjustments
        if name in Upper:
            name = name.upper()
    # Append the first name to the return variable and clear the name variable for the last name
    ret.append(name)
    name = []
    hyphenated = []

    #^ -- Get Last Name -- ^#
    #  If the name is hyphenated, run the last name randomizer twice
    if (random.randrange(100000) / 1000.0) < last_hyphen_percentage:
        hyphenated.append(random_name(1))
        hyphenated.append(random_name(1))
        #  Rerun if more than one hyphen
        hyphenated = check_hyphenated_grammar(hyphenated, 1)
        #  Check for grammar adjustments and form name in name array
        for t in hyphenated:
            if t in Upper:
                name.append(t.upper())
            else:
                name.append(t)
        name = '-'.join(name)
    #  If no hyphen is needed, run the last name randomizer once
    else:
        name = random_name(1)
        if name in Upper:
            name = name.upper()
    # Append the last name to the return variable and clear the name variable for the last name
    ret.append(name)
    name = []

    #^ -- Get Suffix -- ^#
    #  If the name has a suffix, run the suffix randomizer once
    if (random.randrange(100000) / 1000.0) < suffix_hyphen_percentage:
        name = random_name(2)
        if name in Upper:
            name = name.upper()
        ret.append(name)

    #  Return final name joined by spaces
    return ' '.join(ret)



In [68]:
#  Loading the data
draftclass2025 = pd.read_excel('C:/Users/pensh/Desktop/VSCode/DataBase/Data/Draft/Beast2025.xlsx', sheet_name='Combine')
draftclass2025['Name'] = draftclass2025['Name'].str.title()
nameCol = draftclass2025['Name'].str.split(' ')

#  Initiate/Reset other variables
initiate_vars()

#  Does the math to get the weighted name dictionaries
for i in nameCol:
    if len(i) == 2:
        first_name(i[0])
        last_name(i[1])
    elif len(i) == 3:
        first_name(i[0])
        #TODO: This does not work
        if i[1] in double_name:
            last_name(' '.join(i[1:3]))
        else:
            last_name(i[1])
            suffix(i[2])

#  Creates all the names
names_list = []
for i in range(0, 10000):
    names_list.append(get_name(len(nameCol)))

# for i in names_list:
#     print(i)
sorted_list = sorted(names_list, key=len, reverse=False)
for i in sorted_list:
    print(i)

Joe Sy
Bam Kim
O.J. Sy
JP Doss
RJ Blue
MJ Goff
Joe Ojo
RJ Erby
LV Sain
Reid Sy
Roc Ojo
BJ Ford
Irv Lux
Hal Ojo
Joe Fox
KD Wang
TJ Rhea
Bo Ford
JP Reid
OC Gant
Zy Mead
Ed Bone
EJ Ward
Voi Yao
MJ Grier
Day Gant
Drew Lee
Jig West
TJ Ergle
Trey Udo
Nohl Bey
Lake Fox
Kasey Sy
Dan Rowe
Jai Card
PJ Bauer
Rod Epps
Abe Mott
Ken Webb
Irv Epps
LV Diggs
Gus Vann
Tai Ortt
Noa Cook
Day Rowe
Pat Cade
Cam Love
Gage Aka
Bo Ivory
Ed Zimos
Tuni Cox
Zak Paul
Pat Boyd
Day Wehr
Noa Sohn
AT Mausi
Esa Hale
Kai Cade
DJ Peele
Avi Neal
Zak Loop
Zak Pitz
LD Lundy
Ife Dunn
Bru Gums
RJ Payne
KT Ragin
DJ Abdur
Irv Tasi
Rod Boby
Fred Lay
Greg Bey
EJ Coons
AT Capra
Day Jean
Neto Bly
Marc Yao
Sal Pitz
DK Gomes
Cain Van
Obi Patu
Pat Hall
Deco Noa
Sam Webb
JB Fogle
Tai Gaes
Bo Dyson
Tk Downs
Coco Aka
Adam Kim
Tre Rice
Roc Loop
Ira Best
Ozzy Bey
Carl Udo
JP Downs
Kori Ham
Erek Noa
Gage Alo
Taji Roy
KJ Ekeji
TJ Kempf
Hank Ais
OC Bench
JP Afful
Gus Saad
KJ Minor
OC Lucas
Thor Udo
Dez Cobb
Voi Lang
Gio Hill
Ira Pole
Tai Ball

In [67]:
x = []
for i in nameCol:
    x.append(i[1])
x = list(set(x))
x.sort()
print(x)

y = list(last_count.keys())
y.sort()
print(y)

['Abdullah', 'Abdur-Rahman', 'Abdus-Salaam', 'Abolarinwa', 'Abrams', 'Adair', 'Adams', 'Addison', 'Addo', 'Adebayo', 'Adeyi', 'Adkins', 'Adoga', 'Afari', 'Afful', 'Ageli', 'Agina', 'Ahoaﬁ-Noa', 'Ais', 'Aka', 'Akharaiyi', 'Akingbesote', 'Akuchie', 'Alaniz', 'Albert', 'Alexander', 'Alfanostjohn', 'Alford', 'Allen', 'Almendares', 'Alo', 'Alston', 'Alualu', 'Alward', 'Ambush', 'Amos', 'Andersen', 'Anderson', 'Anderson-Taylor', 'Andrews', 'Angry', 'Annoor', 'Antoine', 'Anyanwu', 'Armstead', 'Armstrong', 'Arnold', 'Arrell', 'Arrington', 'Arroyo', 'Artopoeus', 'Asante', 'Asbury', 'Ashby', 'Atkins', 'Atkinson', 'Attafah', 'Atwood', 'Augustin', 'Austin', 'Ayers', 'Ayomanor', 'Azema', 'Bachmeier', 'Badger', 'Bagnah', 'Bailey', 'Baker', 'Baldwin', 'Balkcom', 'Ball', 'Ballentine', 'Ballungay', 'Bandes', 'Bangura', 'Banks', 'Barber', 'Barden', 'Barlage', 'Barlow', 'Barnes', 'Barnett', 'Baron', 'Barquet', 'Barr', 'Barron', 'Barrow', 'Bartholomew', 'Bartlett', 'Barton', 'Baskerville', 'Bass', 'Bassa'